In [ ]:
#실험 구조 요약: 관측 → 다음 회로 결정


import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import matplotlib.pyplot as plt
from collections import Counter

# 기본 설정
sim = Aer.get_backend('qasm_simulator')
shots = 1024
n = 3  # 큐비트 수
steps = 30

# 초기 상태
U = np.random.uniform(0, np.pi, size=n)

# 비트스트링 → RY 각도로 변환 (0 → 0, 1 → π)
def bitstring_to_angles(bitstring):
    return np.array([np.pi if b == '1' else 0.0 for b in bitstring[::-1]])  # Qiskit 순서 보정

# 회로 실행
def run_circuit(U):
    qc = QuantumCircuit(n, n)
    for i in range(n):
        qc.ry(U[i], i)
    qc.barrier()
    qc.measure(range(n), range(n))
    result = execute(qc, sim, shots=shots).result()
    return result.get_counts()

# 기록
trajectory = []
dominant_path = []

for step in range(steps):
    counts = run_circuit(U)
    trajectory.append(counts)

    # 가장 많이 나온 관측 결과를 다음 회로 각도로 변환
    most_common = max(counts.items(), key=lambda x: x[1])[0]
    dominant_path.append(most_common)
    U = bitstring_to_angles(most_common)

# 시각화: dominant state trajectory
def plot_dominant_trajectory(path):
    int_values = [int(b, 2) for b in path]
    plt.figure(figsize=(10, 4))
    plt.plot(int_values, marker='o')
    plt.title("Dominant Measured State Trajectory (as Integer)")
    plt.xlabel("Step")
    plt.ylabel("State (0-7)")
    plt.grid(True)
    plt.show()

plot_dominant_trajectory(dominant_path)
